# Creating and querying the NCBI taxonomy database in SQL

The [NCBI Taxonomy Database](https://www.ncbi.nlm.nih.gov/guide/taxonomy/) gets continuously updated and is available for download.

Download one of the `taxdmp` or `taxdump` files ending in `.Z`, `.tar.gz`, or `.zip`, then uncompress and expand the archive depending on which one you choose. For `.zip`, you can simply use
```bash
unzip taxdmp.zip
```
or for `.tar.gz` you would use the following (`--directory` puts the content into the given directory, so as to keep the current directory clean):
```bash
tar zxf taxdump.tar.gz --directory taxdump
```

The content of the download includes a `readme.txt` file that documents the content and columns of each file.

We'll create a relational database from this download that instantiates the following simple model for a tree (nodes with a label, where each node points to its parent node through a foreing key):

```mermaid
erDiagram
    Node |o--o{ Node : "parent node"
    Node {
      integer ID PK
      string  Label
      string  Rank
      integer Parent_ID FK
    }
```
We include a taxonomic Rank because this can be useful for queries (for example, _find all families of vertebrates_, where _family_ is a taxonomic rank).

## Wrangle data

The need for data wrangling is ubiquitous in data science, and the NCBI taxonomy is a good example. For example, as is, it's very hard or impossible to use database built-in methods for importing tabular data (such as `.import` in the SQLite CLI) due to the use Sybase's native export format. Also, if our application doesn't call for all kinds of different kinds of names for the same node, having them is a nuisance in the best case, and can easily be time-consuming or a source of error.

Fortunately, we can utilize a number of different tools to wrangle the data into shape. Here we use Python. 

In [1]:
import pandas as pd

### Nodes

We'll use `pandas.read_csv`, but need to customize it for the following issues:
- The column separator is multiple characters (`\t|\t`), and the row delimiter is, too (`\t\n`). We handle both by using the regular expression `\t\|\t?` (note escaping the `|`) as column delimiter. (This entails using `engine="python"`.)
- We only need the first three columns; the rest are for entirely unrelated use-cases.
- There are no column headers, and even if there were, we'd probably want to use our own column names.
- We don't want Pandas to use the first column as its index column.

In [2]:
df_nodes = pd.read_csv("taxdump/nodes.dmp", sep="\t\|\t?", names=["ID", "Parent_ID", "Rank"], 
                       header=None, index_col=False, usecols=[0, 1, 2], engine="python")

In [3]:
df_nodes

,ID,Parent_ID,Rank
0,1,1,no rank
1,2,131567,superkingdom
2,6,335928,genus
3,7,6,species
4,9,32199,species
...,...,...,...
2611866,3347144,2729152,species
2611867,3347145,2729152,species
2611868,3348909,1292317,species
2611869,3348910,51988,genus


### Names

Similar format issues for the names table, but here we'll keep all four columns.

In [4]:
df_names = pd.read_csv("taxdump/names.dmp", sep="\t\|\t?", names=["Node_ID", "Label", "unique_name", "name_class"],
                       header=None, index_col=False, engine="python")

In [5]:
df_names

,Node_ID,Label,unique_name,name_class
0,1,all,NaN,synonym
1,1,root,NaN,scientific name
2,2,Bacteria,Bacteria <bacteria>,scientific name
3,2,bacteria,NaN,blast name
4,2,"""Bacteria"" Cavalier-Smith 1987",NaN,authority
...,...,...,...,...
4123352,3348911,MNHN-JL351,MNHN-JL351 <holotype>,type material
4123353,3348911,MNHN:JL351,MNHN:JL351 <holotype>,type material
4123354,3348911,MNHNJL351,MNHNJL351 <holotype>,type material
4123355,3348911,"Vermiviatum covidum (Justine, Gastineau, Gros,...",NaN,authority


We can already see that there seem to be multiple names, each of a different "name class".

### Write to database

We will do the remaining data wrangling in SQL, so first need to write the two dataframes to tables in our database.

We could also do more data wrangling in Python; judge for yourself how easy or not those remaining steps would be in Pandas.

In [6]:
import sqlalchemy

We'll use (and create if it didn't exist) a database on the filesystem. (If you want to use an absolute path, remember that you would have 4 slashes, not 3: `sqlite:////path/to/your/ncbi-tax.db`.

In [7]:
db = sqlalchemy.create_engine("sqlite:///ncbi-tax.db").connect()

We'll drop the tables if they already exist; use a different database if you don't want this behavior. Also, we don't care about persisting Pandas dataframe index, so we leave it out.

In [8]:
df_nodes.to_sql("ncbi_nodes", con=db, if_exists='replace', index=False)

2611871

In [9]:
df_names.to_sql("ncbi_names", con=db, if_exists='replace', index=False)

4123357

We can check that the number of affected rows reported by the two statements matches the length of the respective dataframes.

And we should now be done with the Python part, so close the connection.

In [10]:
db.close()

### Wrangling the data into shape using SQL

First, load the SQL extension and connect to the database we just created.

In [11]:
%load_ext sql

In [12]:
%sql sqlite:///ncbi-tax.db

Connecting to 'sqlite:///ncbi-tax.db'

Let's check that the tables are there as expected:

In [13]:
%sql select * from ncbi_nodes limit 3;

Running query in 'sqlite:///ncbi-tax.db'

ID,Parent_ID,Rank
1,1,no rank
2,131567,superkingdom
6,335928,genus


In [14]:
%sql select * from ncbi_names limit 3;

Running query in 'sqlite:///ncbi-tax.db'

Node_ID,Label,unique_name,name_class
1,all,None,synonym
1,root,None,scientific name
2,Bacteria,Bacteria,scientific name


The nodes and names tables have essentially the following relationship:
```mermaid
erDiagram
    ncbi_nodes ||--|{ ncbi_nodes : "parent node"
    ncbi_nodes ||--o{ ncbi_names : "has name"
    ncbi_nodes {
      integer ID PK
      integer Parent_ID FK
      string  Rank
    }
    ncbi_names {
      integer Node_ID FK
      string  Label
      string  unique_name
      string  name_class
    }
```
We can get an idea about the names to node relationship by looking at _Homo sapiens_. (Note that NCBI taxonomy IDs are "persistent"; that is, they don't change from one release to another, provided the taxon doesn't change. Hence, 9606 for _Homo sapiens_ will still be the same in years from now.)

In [15]:
%%sql
SELECT * from ncbi_names
WHERE Node_ID = 9606

Running query in 'sqlite:///ncbi-tax.db'

Node_ID,Label,unique_name,name_class
9606,"Homo sapiens Linnaeus, 1758",None,authority
9606,Homo sapiens (PARIS),None,includes
9606,Homo sapiens,None,scientific name
9606,Homo sapiens (SIRT6),None,includes
9606,human,None,genbank common name


In our desired model, we are only interested in one label for each node. It seems the best choice for this is the scientific name, so we'll need to join ncbi_nodes and ncbi_names. To facilitate this, we'll index the columns we'll presumably have to use:

In [16]:
%%sql
CREATE INDEX ncbi_id ON ncbi_nodes(ID);
CREATE INDEX ncbi_nodeid ON ncbi_names(Node_ID);
CREATE INDEX ncbi_nameclass ON ncbi_names (name_class);

Running query in 'sqlite:///ncbi-tax.db'

++
||
++
++

We should check that using _scientific name_ as "name class" is indeed well suitable, i.e., whether our assumption that there is at most _one_ such label per node is true.

In [17]:
%%sql
SELECT Node_id, group_concat(Label, ', ') AS Labels
FROM ncbi_names
WHERE name_class = 'scientific name'
GROUP by Node_id
HAVING COUNT(Label) > 1;

Running query in 'sqlite:///ncbi-tax.db'

Node_ID,Labels


Conversely:

In [18]:
%%sql
SELECT COUNT(Node_ID)
FROM (
    SELECT Node_ID
    FROM ncbi_names
    WHERE name_class = 'scientific name'
    GROUP by Node_id
    HAVING COUNT(Label) = 1
)

Running query in 'sqlite:///ncbi-tax.db'

COUNT(Node_ID)
2611871


This is in fact the same number of nodes as the nodes table, so apparently every node has exactly one scientific name!

Using this, we can create the desired `Node` table using a straight inner join:

In [21]:
%%sql
CREATE TABLE Node AS
    SELECT n.ID, n.Parent_ID, nm.Label, n.rank 
    FROM ncbi_nodes as n INNER JOIN ncbi_names as nm on (n.id = nm.node_id)
    WHERE nm.name_class = 'scientific name';

Running query in 'sqlite:///ncbi-tax.db'

++
||
++
++

### Checks and finalize

Let's first check whether we have the right number of rows and that they match our expectations:

In [22]:
%sql SELECT count(*) from Node

Running query in 'sqlite:///ncbi-tax.db'

count(*)
2611871


In [24]:
%sql select * from Node;

Running query in 'sqlite:///ncbi-tax.db'

ID,Parent_ID,Label,Rank
1,1,root,no rank
2,131567,Bacteria,superkingdom
6,335928,Azorhizobium,genus
7,6,Azorhizobium caulinodans,species
9,32199,Buchnera aphidicola,species
10,1706371,Cellvibrio,genus
11,1707,Cellulomonas gilvus,species
13,203488,Dictyoglomus,genus
14,13,Dictyoglomus thermophilum,species
16,32011,Methylophilus,genus


Create indexes and enforce uniqueness:

In [27]:
%%sql
SELECT Label, GROUP_CONCAT(ID, ', ') AS Nodes
FROM Node
GROUP BY Label
HAVING Count(ID) > 1

Running query in 'sqlite:///ncbi-tax.db'

Label,Nodes
Abietinella,"61533, 655744"
Abroma,"82453, 1745610"
Abronia,"76641, 76710"
Abronia gadovii,"3147712, 3162976"
Abrus,"3815, 2664681"
Acanthella,"85782, 1160307"
Acanthocarpus,"59090, 1844874"
Acanthocephala,"10232, 2316800"
Acanthocephalus,"185725, 2605606"
Acanthococcus,"1443289, 2742496"


So scientific name as Label is apparently not necessarily unique (even though for most it presumably is). 

In [33]:
%%sql
CREATE UNIQUE INDEX IF NOT EXISTS NODE_ID on Node (ID);
CREATE INDEX IF NOT EXISTS Parent_id on Node(Parent_iD);
CREATE INDEX IF NOT EXISTS  label_idx on Node(Label);

Running query in 'sqlite:///ncbi-tax.db'

++
||
++
++

We aren't interested in bacteria and viruses, whose trees can potentially be messy (horizontal gene transfer etc), so we artificially set the root at _Eukaryota_ by setting the Parent_ID of that node to NULL.

In [34]:
%sql select * from node where label = 'Eukaryota';

Running query in 'sqlite:///ncbi-tax.db'

ID,Parent_ID,Label,Rank
2759,131567,Eukaryota,superkingdom


In [35]:
%sql UPDATE Node SET Parent_id = NULL WHERE id = 2759;

Running query in 'sqlite:///ncbi-tax.db'

1 rows affected.

++
||
++
++

In [36]:
%sql select * from node where label = 'Eukaryota';

Running query in 'sqlite:///ncbi-tax.db'

ID,Parent_ID,Label,Rank
2759,None,Eukaryota,superkingdom


## Querying the database

Now we're ready to try and play with queries and CTEs. Remember that CTEs are in essence temporary tables created "on the fly" for _this_ query. They don't have to be recursive, and we can break down recursive CTEs by first making them non-recursive and checking that our initial non-recursive results (= "starting rows") are correct.

In [38]:
%config SqlMagic.displaylimit = 0

In [42]:
%%sql
-- non-recursive at first
WITH Node_Path (Node_ID, Ancestor_ID, Path_Len) AS (
       SELECT ID, Parent_ID, 1 -- Initial Subquery
       FROM   Node
       WHERE  Label = 'Homo sapiens'
)
--  non-recursive we should one level of ancestry (namely parents)
SELECT n.ID, n.Label, a.ID, a.Label, np.Path_Len
FROM Node_Path np JOIN Node AS n ON (np.Node_ID = n.ID)
     JOIN Node AS a ON (np.Ancestor_ID = a.ID)
ORDER BY np.Path_Len;

Running query in 'sqlite:///ncbi-tax.db'

ID,Label,ID_1,Label_1,Path_Len
9606,Homo sapiens,9605,Homo,1


In [41]:
%%sql
-- non-recursive at first, multiple starting rows
WITH Node_Path (Node_ID, Ancestor_ID, Path_Len) AS (
       SELECT ID, Parent_ID, 1 -- Initial Subquery
       FROM   Node
       WHERE  Label IN ('Homo sapiens', 'Gallus gallus', 'Canis lupus')
)
SELECT n.ID, n.Label, a.ID, a.Label, np.Path_Len
FROM Node_Path np JOIN Node AS n ON (np.Node_ID = n.ID)
     JOIN Node AS a ON (np.Ancestor_ID = a.ID)
ORDER BY np.Path_Len;

Running query in 'sqlite:///ncbi-tax.db'

ID,Label,ID_1,Label_1,Path_Len
9612,Canis lupus,9611,Canis,1
9031,Gallus gallus,9030,Gallus,1
9606,Homo sapiens,9605,Homo,1


**The recursive part of the query is then added to the initial subquery through a `UNION ALL`.**

To recursively extend (= add rows to) the Node_Path towards the root, we hold the starting point(s) constant (i.e., we report for recursive rows the same Node_ID as for the paths we already have), and then extend the path(s) we have by reporting the _parent(s)_ of the ancestors we already have, incrementing the path lengths by one. 

In [44]:
%%sql
WITH RECURSIVE Node_Path (Node_ID, Ancestor_ID, Path_Len) AS (
       SELECT ID, Parent_ID, 1 -- Initial Subquery
       FROM   Node
       WHERE  Label = 'Homo sapiens'
       UNION ALL
       -- Recursive Subquery
       SELECT Node_ID, p.Parent_ID, Path_Len + 1
       FROM   Node AS p JOIN Node_Path ON (p.ID = Node_Path.Ancestor_ID)
)
SELECT n.ID, n.Label, a.ID, a.Label, np.Path_Len
FROM Node_Path np JOIN Node AS n ON (np.Node_ID = n.ID)
     JOIN Node AS a ON (np.Ancestor_ID = a.ID)
ORDER BY np.Path_Len;

Running query in 'sqlite:///ncbi-tax.db'

ID,Label,ID_1,Label_1,Path_Len
9606,Homo sapiens,9605,Homo,1
9606,Homo sapiens,207598,Homininae,2
9606,Homo sapiens,9604,Hominidae,3
9606,Homo sapiens,314295,Hominoidea,4
9606,Homo sapiens,9526,Catarrhini,5
9606,Homo sapiens,314293,Simiiformes,6
9606,Homo sapiens,376913,Haplorrhini,7
9606,Homo sapiens,9443,Primates,8
9606,Homo sapiens,314146,Euarchontoglires,9
9606,Homo sapiens,1437010,Boreoeutheria,10


To recursively extend the Node_Path towards the leaf nodes (= report descendants), we hold the starting ancestor constant (i.e., we report for recursive rows the same ancestor as for the paths we already have), and then extend the path(s) we have by reporting as the next descendent the node(s) for which the Node_ID of the paths we already have is the parent, incrementing the path lengths by one.

In [53]:
%%sql
WITH RECURSIVE Node_Path (Node_ID, Ancestor_ID, Path_Len) AS (
       SELECT ID, Parent_ID, 1 -- Initial Subquery
       FROM   Node
       WHERE  Parent_ID = (SELECT ID FROM Node WHERE Label = 'Hominoidea')
       UNION ALL
       -- Recursive Subquery
       SELECT d.ID, Ancestor_ID, Path_Len + 1
       FROM   Node AS d JOIN Node_Path ON (d.Parent_ID = Node_Path.Node_ID)
)
SELECT n.ID, n.Label, n.Rank, a.ID, a.Label, np.Path_Len
FROM Node_Path np JOIN Node AS n ON (np.Node_ID = n.ID)
     JOIN Node AS a ON (np.Ancestor_ID = a.ID)
-- we'll keep it to clean ranks and taxa
WHERE n.Rank != 'no rank' AND n.Label NOT LIKE '% sp.%' AND n.Label NOT LIKE '% x %'
ORDER BY np.Path_Len;

Running query in 'sqlite:///ncbi-tax.db'

ID,Label,Rank,ID_1,Label_1,Path_Len
9577,Hylobatidae,family,314295,Hominoidea,1
9604,Hominidae,family,314295,Hominoidea,1
9578,Hylobates,genus,314295,Hominoidea,2
207598,Homininae,subfamily,314295,Hominoidea,2
325165,Nomascus,genus,314295,Hominoidea,2
325166,Symphalangus,genus,314295,Hominoidea,2
325167,Hoolock,genus,314295,Hominoidea,2
607660,Ponginae,subfamily,314295,Hominoidea,2
9579,Hylobates agilis,species,314295,Hominoidea,3
9580,Hylobates lar,species,314295,Hominoidea,3


### Common ancestors

To find common ancestors, we start the recursive extending of paths from multiple starting rows (namely those we want to find common ancestors for).

In [57]:
%%sql
WITH RECURSIVE Node_Path (Node_ID, Ancestor_ID, Path_Len) AS (
       SELECT ID, Parent_ID, 1 -- Initial Subquery
       FROM   Node
       WHERE  Label IN ('Homo sapiens','Gallus')
       UNION ALL
       -- Recursive Subquery
       SELECT Node_ID, p.Parent_ID, Path_Len + 1
       FROM   Node AS p JOIN Node_Path ON (p.ID = Node_Path.Ancestor_ID)
)
SELECT n.ID, n.Label, a.ID, a.Label, np.Path_Len
FROM Node_Path np JOIN Node AS n ON (np.Node_ID = n.ID)
     JOIN Node AS a ON (np.Ancestor_ID = a.ID)
ORDER BY np.Path_Len;

Running query in 'sqlite:///ncbi-tax.db'

ID,Label,ID_1,Label_1,Path_Len
9030,Gallus,9072,Phasianinae,1
9606,Homo sapiens,9605,Homo,1
9030,Gallus,9005,Phasianidae,2
9606,Homo sapiens,207598,Homininae,2
9030,Gallus,8976,Galliformes,3
9606,Homo sapiens,9604,Hominidae,3
9030,Gallus,1549675,Galloanserae,4
9606,Homo sapiens,314295,Hominoidea,4
9030,Gallus,8825,Neognathae,5
9606,Homo sapiens,9526,Catarrhini,5


As we can see, some ancestors appear only once (the ones unique to each starting node), and some ancestors appear twice. The latter are ones in common between the starting nodes. The easiest way to report only the ones in common is to group by ancestor. The groups with more than one starting node must be the common ancestors (and conversely, the groups with only one starting node are the ancestor not in common).

In [61]:
%%sql
WITH RECURSIVE Node_Path (Node_ID, Ancestor_ID, Path_Len) AS (
       SELECT ID, Parent_ID, 1 -- Initial Subquery
       FROM   Node
       WHERE  Label IN ('Homo sapiens','Gallus gallus')
       UNION ALL
       -- Recursive Subquery
       SELECT Node_ID, p.Parent_ID, Path_Len + 1
       FROM   Node AS p JOIN Node_Path ON (p.ID = Node_Path.Ancestor_ID)
)
SELECT a.ID, a.Label, MAX(np.Path_Len) AS Max_Path_Len
FROM Node_Path np JOIN Node AS n ON (np.Node_ID = n.ID)
     JOIN Node AS a ON (np.Ancestor_ID = a.ID)
-- group by ancestor
GROUP BY a.ID, a.Label
-- groups with more than one starting node are the common ancestors
-- change this to = 1 to see the ones not in common
HAVING COUNT(n.ID) > 1
ORDER BY np.Path_Len;

Running query in 'sqlite:///ncbi-tax.db'

ID,Label,Max_Path_Len
32524,Amniota,16
32523,Tetrapoda,17
1338369,Dipnotetrapodomorpha,18
8287,Sarcopterygii,19
117571,Euteleostomi,20
117570,Teleostomi,21
7776,Gnathostomata,22
7742,Vertebrata,23
89593,Craniata,24
7711,Chordata,25


This works with any number of starting rows (though we have adapt the condition for which groups are reported):

In [62]:
%%sql
WITH RECURSIVE Node_Path (Node_ID, Ancestor_ID, Path_Len) AS (
       SELECT ID, Parent_ID, 1 -- Initial Subquery
       FROM   Node
       WHERE  Label IN ('Homo sapiens','Gallus gallus','Apis mellifera')
       UNION ALL
       -- Recursive Subquery
       SELECT Node_ID, p.Parent_ID, Path_Len + 1
       FROM   Node AS p JOIN Node_Path ON (p.ID = Node_Path.Ancestor_ID)
)
SELECT a.ID, a.Label, MAX(np.Path_Len) AS Max_Path_Len
FROM Node_Path np JOIN Node AS n ON (np.Node_ID = n.ID)
     JOIN Node AS a ON (np.Ancestor_ID = a.ID)
-- group by ancestor
GROUP BY a.ID, a.Label
-- groups with more than two starting node are the common ancestors
-- change this to = 1 to see the ones not in common
HAVING COUNT(n.ID) > 2
ORDER BY np.Path_Len;

Running query in 'sqlite:///ncbi-tax.db'

ID,Label,Max_Path_Len
33213,Bilateria,27
6072,Eumetazoa,28
33208,Metazoa,29
33154,Opisthokonta,30
2759,Eukaryota,31


Closing sqlite:///ncbi-tax.db